In [4]:
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import pandas as pd
from IPython.display import clear_output
from skimage.segmentation import find_boundaries
import os
import sys
import importlib
from skimage import morphology
from skimage.segmentation import find_boundaries
import trimesh
import porespy as ps
import h5py

In [2]:
sys.path.insert(1, "/Users/dvarelat/Documents/MASTER/TFM/methods")
import cardiac_region
importlib.reload(cardiac_region)
import cardiac_region as c

In [6]:
cellpose = "/Users/dvarelat/Documents/MASTER/TFM/DATA/RESULTS/nuclei/20190404_E2_MASK_EQ_XYZ_decon.nii.gz"
DFFILE = f"/Users/dvarelat/Documents/MASTER/TFM/DATA/EXTRACTION/7_20190404_E2/cell_properties.csv"
mesh_myo = "/Users/dvarelat/Documents/MASTER/TFM/lines_ply/line_20190404_E2_myo_10000.ply" #"/homedtic/dvarela/lines_ply/line_20190404_E2_myo_10000.ply"
mesh_spl = "/Users/dvarelat/Documents/MASTER/TFM/lines_ply/line_20190404_E2_spl_10000.ply" #"/homedtic/dvarela/lines_ply/line_20190404_E2_spl_10000.ply"
gasp_mem = f"/Users/dvarelat/Documents/MASTER/TFM/DATA/RESULTS/membranes/GASP_PNAS/20190404_E2_mGFP_XYZ_predictions_GASP.nii.gz"
sem = f"/Users/dvarelat/Documents/MASTER/TFM/DATA/RESULTS/membranes/PNAS/20190404_E2_mGFP_CardiacRegion_0.5_ZXY_predictions.h5"
linefile = F"/Users/dvarelat/Documents/MASTER/TFM/DATA/LINES/line_20190404_E2.nii.gz"
decon = "/Users/dvarelat/Documents/MASTER/TFM/DATA/DECON_05/MGFP/20190404_E2_mGFP_decon_0.5.nii.gz"

In [8]:
pred_zxy = np.array(h5py.File(sem, "r")["predictions"])
print(pred_zxy.shape)
pred_xyz = np.swapaxes(np.swapaxes(pred_zxy[0, :, :, :], 0, 2), 0, 1)
decon05 = nib.load(decon).get_fdata()
decon05 = decon05[:, :, :, 0]

(1, 432, 1020, 764)


In [9]:
margs = c.crop_line(linefile, decon, escala2048=False, ma=5)
cut = c.crop_embryo(margs, decon)
zeros = np.zeros(decon05.shape)
zeros[margs[0][0] : margs[1][0], margs[0][1] : margs[1][1], margs[0][2] : margs[1][2]] = pred_xyz

MARGIN : 5
Line information
{'x_size': 1024, 'y_size': 1024, 'z_size': 448, 'x_res': 0.757206, 'y_res': 0.757206, 'z_res': 0.99691}
LINE SHAPE = (1024, 1024, 448)
SCALING WITH DECON 05 --> 1024
{'x_size': 1024, 'y_size': 1024, 'z_size': 448, 'x_res': 0.757206, 'y_res': 0.757206, 'z_res': 0.99691}
CROP SHAPE = (1020, 764, 432, 1)


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [14]:
c.saveNifti((zeros > 0.5).astype("uint16"),
            c.load3D_metadata(decon), 
            f"/Users/dvarelat/Documents/MASTER/TFM/DATA/RESULTS/membranes/PNAS/20190404_E2_mGFP_0.5_XYZ_predictions.nii.gz"
           )

In [4]:
myo = trimesh.load_mesh(mesh_myo, process=False)
spl = trimesh.load_mesh(mesh_spl, process=False)
df = pd.read_csv(DFFILE)

In [5]:
pred_nu = nib.load(cellpose).get_fdata()
print(pred_nu.shape)

(1024, 1024, 448)


In [6]:
pred_mem = nib.load(gasp_mem).get_fdata()
print(pred_mem.shape)

(1024, 1024, 448)


In [7]:
im = find_boundaries(pred_nu).astype("uint16")

In [8]:
c.saveNifti(im,
            c.load3D_metadata(cellpose), 
            f"/Users/dvarelat/Documents/MASTER/TFM/FIGURES/20190404_E2_cellpose_boundaries.nii.gz")

In [15]:
c.load3D_metadata(cellpose)["z_res"]*180

179.44379568099976

In [10]:
df.shape

(7826, 15)

In [11]:
dim_info = c.load3D_metadata(gasp_mem)
vertices_location_spl = np.floor(spl.vertices / [dim_info["x_res"], dim_info["y_res"], dim_info["z_res"]]).astype("uint16")
vertices_labels_spl = pred_mem[vertices_location_spl[:, 0], vertices_location_spl[:, 1], vertices_location_spl[:, 2]]


In [12]:
vertices_location_myo = np.floor(myo.vertices / [dim_info["x_res"], dim_info["y_res"], dim_info["z_res"]]).astype("uint16")
vertices_labels_myo = pred_mem[vertices_location_myo[:, 0], vertices_location_myo[:, 1], vertices_location_myo[:, 2]]


In [13]:
myo.vertices.shape

(5002, 3)

In [16]:
indices_lines5_spl = list(df[df.lines == 5].cell_in_props.values)
indices_vertices_surface_spl = list(df[df.original_labels.isin(vertices_labels_spl)].cell_in_props.values)

indices_lines5_myo = list(df[df.lines == 1].cell_in_props.values)
indices_vertices_surface_myo = list(df[df.original_labels.isin(vertices_labels_myo)].cell_in_props.values)

In [17]:
indices_vertices_surface_spl = [i for i in indices_vertices_surface_spl if i not in indices_vertices_surface_myo]
#set(indices_vertices_surface_spl).intersection(set(indices_vertices_surface_myo))

In [18]:
df["lines2"] = df["cell_in_props"].apply(lambda x: 5 if x in indices_vertices_surface_spl else (1 if x in indices_vertices_surface_myo else 0))



In [19]:
df.groupby(["lines2"]).count()

,cell_in_props,volumes,sphericities,original_labels,centroids,lines,axis_major_length,axis_minor_length,nuclei_label_cent,nuclei_cell_in_props,eigenvalues,eigenvectors,eccentricity3d,solidity,feret_diameter_max
lines2,,,,,,,,,,,,,,,
0,5260,5260,5260,5260,5260,5260,5260,5260,5260,5260,5260,5260,5260,5260,5260
1,1459,1459,1459,1459,1459,1459,1459,1459,1459,1459,1459,1459,1459,1459,1459
5,1107,1107,1107,1107,1107,1107,1107,1107,1107,1107,1107,1107,1107,1107,1107


In [20]:
cells_labels = list(df.lines2)
props_mem = ps.metrics.regionprops_3D(morphology.label(pred_mem))

In [25]:
im = np.zeros(shape=pred_mem.shape)
for i,cell in enumerate(df.cell_in_props):
    r = props_mem[cell]
    mask = r.image
    temp = mask * cells_labels[i]
    s = ps.tools.bbox_to_slices(r.bbox)
    im[s] += temp

In [26]:
c.saveNifti(im,
            c.load3D_metadata(cellpose), 
            f"/Users/dvarelat/Documents/MASTER/TFM/FIGURES/20190404_E2_myoandspl.nii.gz")

In [46]:
FOLDERS = [
    "1_20190504_E1",
    "2_20190520_E2",
    "3_20190516_E3",
    "4_20190806_E3",
    "5_20190401_E3",
    '6_20190401_E1',
    "7_20190404_E2",
    "8_20190517_E1",
    "9_20190520_E1",
    "10_20190520_E4",
]
list_dfs_radio =[]
for i,folder in enumerate(FOLDERS):
    print(folder)
    e = folder.split("_")[1] + "_" + folder.split("_")[2]
    fr = f"/Users/dvarelat/Documents/MASTER/TFM/DATA/EXTRACTION/{folder}/{e}_cell_properties_radiomics.csv"
    df = pd.read_csv(f"/Users/dvarelat/Documents/MASTER/TFM/DATA/EXTRACTION/NUCLEI/{e}_NUCLEI_properties.csv")
    dfr= pd.read_csv(fr)
    print(dfr.shape)
    print(dfr[dfr.nuclei_cell_in_props != -1].shape[0] / dfr.shape[0])
    print(len(set(df.original_labels.values).intersection(set([i for i in dfr.nuclei_label_cent.values if i != -1]))))
    nucleis = len(set(df.original_labels.values).intersection(set([i for i in dfr.nuclei_label_cent.values if i != -1])))
    print(nucleis/df.shape[0])
    print("-------------")

1_20190504_E1
(2637, 68)
0.8668941979522184
1806
0.4898291293734744
-------------
2_20190520_E2
(3299, 68)
0.9054258866323128
718
0.24921902117320374
-------------
3_20190516_E3
(7865, 68)
0.7327399872854419
5028
0.5869032333372243
-------------
4_20190806_E3
(5216, 68)
0.8305214723926381
3533
0.5094448449891853
-------------
5_20190401_E3
(5127, 68)
0.8712697483908719
3577
0.47929786948948144
-------------
6_20190401_E1
(4811, 68)
0.7875701517356059
2970
0.44614691302388465
-------------
7_20190404_E2
(7826, 68)
0.7776641962688474
5327
0.5765775516830826
-------------
8_20190517_E1
(7601, 68)
0.8748848835679516
5653
0.570318805488297
-------------
9_20190520_E1
(7739, 68)
0.7937718051427833
5086
0.5393425238600212
-------------
10_20190520_E4
(1532, 68)
0.8289817232375979
997
0.26264488935721814
-------------


In [44]:
len(set(df.original_labels.values).intersection(set([i for i in dfr.nuclei_label_cent.values if i != -1])))

997